## Import utilities

In [ ]:
# Librerie per lettura file
import pandas as pd
from pathlib import Path
import json
import os
import psutil
import numpy as np
from tqdm import tqdm
import warnings
import pickle
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
import optuna
import gc




#Kfold and GridSearch
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV



# Grafici
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

# Data Analysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    precision_recall_curve, classification_report, confusion_matrix,
    roc_curve, roc_auc_score, average_precision_score
)
from imblearn.over_sampling import SMOTE
from sklearn.multioutput import MultiOutputRegressor
from scipy.stats import pearsonr


# Modelli con Alberi
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from lightgbm import LGBMClassifier, LGBMRegressor


# Librerie Torch per MLP
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, SubsetRandomSampler
from torchmetrics import F1Score, Accuracy, Precision, Recall
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

# Explainable AI (SHAP)
import shap

# Librerie per gestione dati parallela
import modin.pandas as mpd
import modin.config as cfg

# Visualizzazione matrice di confusione
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow as tf

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from imblearn.over_sampling import SMOTE
import numpy as np
from tensorflow.keras import layers, models

import tensorflow.keras.backend as K
import psutil
import joblib

In [ ]:
matplotlib.use('Agg') 
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)
optuna.logging.set_verbosity(optuna.logging.ERROR)

In [ ]:
def save_state(save_path, state_dict):
    joblib.dump(state_dict, save_path)

def load_state(save_path):
    if os.path.exists(save_path):
        return joblib.load(save_path)
    else:
        return None


## Reading

In [ ]:
score_df = pd.read_csv("Preprocessing/drug_scores_with_targets.csv")
df_clean = pd.read_csv("Preprocessing/transcrittoma_pulito.csv")

## Model Sensitivity

In this section, you can run models that predict drug sensitivity only. The analysis specifically focuses on the top 30 drugs, although you can include more if desired. However, note that drugs with a negative score perform worse than a random model and are therefore not recommended for use—this applies especially to drugs ranked beyond the 194th position.

The DL functions are implemented here

### Funzione

#### MLP

In [ ]:
def create_mlp_model(input_shape, dim=128, dropout=0.2, learning_rate = 0.01):

    """
    Creates and compiles a simple multi-layer perceptron (MLP) model for binary classification.
    
    The architecture includes three hidden layers with ReLU activation, dropout for regularization, 
    and a final sigmoid output layer. Uses Adam optimizer with specified learning rate.
    
    Returns the model.
    """

    model = models.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    model.add(layers.Dense(dim, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dim//2, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dim//4, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model




def create_objective(X_train, X_val, y_train, y_val):

    """
    Creates an Optuna objective function to optimize hyperparameters for an MLP model.

    The objective trains a model with suggested hyperparameters (layer size, dropout, learning rate),
    evaluates it on validation data, and returns the average precision score as the optimization target.
    
    Returns the objective function to be used in an Optuna study.
    """

    def objective(trial):
        dim = trial.suggest_categorical('dim', [64, 128, 256, 512,1024])
        dropout = trial.suggest_float('dropout', 0.1, 0.6)
        learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True)


        try:

            model = tf.keras.Sequential([
                    tf.keras.Input(shape=(X_train.shape[1],)),
                    tf.keras.layers.Dense(dim, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(dim // 2, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(dim // 4, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(1, activation='sigmoid')
                ])


            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)

            model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    verbose=0,
                    callbacks=[early_stopping])
            y_pred = model.predict(X_val, verbose = 0).ravel()

        except Exception as e:
            print(f"[ERROR] Optuna CNN Trial fallito: {e}")
            return 0.0  

        return average_precision_score(y_val, y_pred)

    return objective


#### CNN

In [ ]:
def create_cnn_model(input_shape, dim=128, learning_rate = 0.01):
    """
    Creates and compiles a 1D convolutional neural network (CNN) model for binary classification.

    The model consists of three Conv1D layers with ReLU activations and max pooling, followed by
    a fully connected layer and a sigmoid output layer. Uses Adam optimizer with given learning rate.

    Returns the compiled CNN model.
    """
    model = models.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    model.add(layers.Conv1D(dim, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(dim//2, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(dim//4, 3, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(dim//8, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
    return model


def create_objective_cnn(X_train, X_val, y_train, y_val):

    """
    Creates an Optuna objective function to optimize hyperparameters for the 1D CNN model.

    The objective trains a CNN with trial-suggested parameters (dim and learning rate),
    performs early stopping, and evaluates performance on validation data using average precision score.

    Skips trials if input length is too short for the CNN architecture.

    Returns the objective function for use in an Optuna study.
    """

    def objective(trial):
        dim = trial.suggest_categorical('dim', [64, 128, 256, 512])
        learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

        input_length = X_train.shape[1]
      
        min_required_length = 3 + 2 + 2  
        if input_length < min_required_length:
            print(f"[SKIP TRIAL] Input shape troppo corta: {input_length} (minimo richiesto: {min_required_length})")
            return 0.0 

        try:
            model = tf.keras.Sequential([
                tf.keras.Input(shape=(input_length, 1)),
                tf.keras.layers.Conv1D(dim, 3, activation='relu'),
                tf.keras.layers.MaxPooling1D(2),
                tf.keras.layers.Conv1D(dim//2, 3, activation='relu'),
                tf.keras.layers.MaxPooling1D(2),
                tf.keras.layers.Conv1D(dim//4, 3, activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(dim//8, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)

            model.fit(X_train, y_train,
                      validation_data=(X_val, y_val),
                      epochs=20,
                      batch_size=32,
                      verbose=0,
                      callbacks=[early_stopping])

            y_pred = model.predict(X_val, verbose=0).ravel()
            return average_precision_score(y_val, y_pred)

        except Exception as e:
            print(f"[ERROR] Optuna CNN Trial fallito: {e}")
            return 0.0 
    return objective


#### Model

In [ ]:
def modello_deep_learning(best_features, train_set, train_cells, rf_params={}, 
               base_models=None, param_grid=None, n_splits=5, random_state=4, model_type = 'mlp'):
    
    """
    Performs k-fold cross-validation training of deep learning models (MLP or CNN) on the given dataset.

    For each fold:
    - Splits data into training and validation based on cell line identifiers.
    - Applies SMOTE to balance classes in training data.
    - Reshapes data as needed for the selected model type.
    - Uses Optuna to optimize hyperparameters (architecture dimensions, dropout, learning rate).
    - Trains the model with early stopping on validation loss.
    - Evaluates and stores performance metrics (ROC AUC, AUC PR, F1 score).
    
    Returns a list of trained models with their metrics, and aggregated true and predicted validation labels.
    """


    # Default model
    if base_models is None:
        base_models = [create_cnn_model for _ in range(n_splits)]  
    
    # Kfold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    models_per_fold = []
    y_val_all, y_val_pred_all = [], []
    i = 0

    for train_idx, val_idx in kf.split(train_cells):
        # Split training and validation
        train_cells_fold = train_cells[train_idx]
        val_cells_fold = train_cells[val_idx]

        train_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(train_cells_fold)]
        val_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(val_cells_fold)]

        X_train = train_fold[best_features.index.tolist()]
        y_train = train_fold["Sensitivity"]

        X_val = val_fold[best_features.index.tolist()]
        y_val = val_fold["Sensitivity"]


        if len(train_fold) == 0 or len(val_fold) == 0:
            print(f"Fold {i}: train or validation fold is empty. Skipping.")
            continue

        # SMOTE
        smote = SMOTE(random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        if model_type == 'cnn':
            X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
            X_val = X_val.values.reshape(-1, X_val.shape[1], 1)
        else: 
            X_train = X_train.values.astype("float32")
            X_val = X_val.values.astype("float32")

        optuna.logging.set_verbosity(optuna.logging.WARNING)

        # Create models and optuna
        if model_type == 'mlp':
            objective = create_objective(X_train, X_val, y_train, y_val)
            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=10,show_progress_bar=False)

            print("Best hyperparameters found:")
            print(study.best_params)

            best_params = study.best_params
            model = create_mlp_model(
                input_shape=(X_train.shape[1],),
                dim=best_params['dim'],
                dropout=best_params['dropout'],
                learning_rate=best_params['lr']
            )  
            
        elif model_type == 'cnn':
            objective = create_objective_cnn(X_train, X_val, y_train, y_val)
            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=20, show_progress_bar=False)

            print("Best hyperparameters found:")
            print(study.best_params)

            best_params = study.best_params
            model = create_cnn_model(
                input_shape=(X_train.shape[1],1),
                dim=best_params['dim'],
                learning_rate=best_params['lr']
            )

        # Fit model
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
        model.fit(X_train, y_train, 
                  epochs=20, 
                  batch_size=min(len(X_train), 32), 
                  verbose=0, 
                  validation_data=(X_val, y_val), 
                  callbacks=[early_stopping])

        # Predict probabilities
        y_pred_prob = model.predict(X_val, verbose = 0)
        y_val_all.extend(y_val)
        y_val_pred_all.extend(y_pred_prob)

        # Metric score
        auc_score = roc_auc_score(y_val, y_pred_prob)
        auc_pr = average_precision_score(y_val, y_pred_prob)
        f1 = f1_score(y_val, (y_pred_prob > 0.5).astype(int))
        models_per_fold.append({
            "model": model,
            "auc_score": auc_score,
            "auc_pr": auc_pr,
            "f1_score": f1
        })

        i += 1

        # Cleaning for memory
        tf.keras.backend.clear_session()
        del model
        gc.collect()


    return models_per_fold, y_val_all, y_val_pred_all


#### Train models

In [ ]:
def train_model(df_clean, shap_dir, l=31, model_type = 'mlp'):

    """
    Trains predictive models for drug sensitivity.
    The function processes multiple drugs in a loop, performing data preprocessing, training, validation, and evaluation 
    for each drug separately.

    Key steps include:
    - Loading a previous training state to allow resuming interrupted training
    - Splitting data into training and test sets stratified by sensitivity
    - Training models with K-fold cross-validation and hyperparameter tuning via Optuna
    - Aggregating results and calculating performance metrics such as AUC-ROC, AUC-PR, and F1-score
    - Generating and saving plots (precision-recall vs cutoff, confusion matrix, ROC and PR curves)
    - Saving intermediate training states for checkpointing and resuming

    Parameters:
    - df_clean: cleaned input dataframe containing features and drug response labels
    - shap_dir: directory path to save results, plots, and model checkpoints
    - l: number of drugs to process (default 31)
    - model_type: specifies the type of deep learning model ('mlp' or 'cnn')


    """
    if not os.path.exists(shap_dir):
        os.makedirs(shap_dir)

    label_encoders = {}
    df_encoded = df_clean.copy()
    df_encoded = df_encoded.dropna(subset=['Sensitivity'])
    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object' or df_encoded[col].dtype.name == 'category':
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le

    state_file = os.path.join(shap_dir, "training_state.pkl")
    state = load_state(state_file)
    if state:
        print("Previous state found. Resuming from where I left off.")
        models_results_rf_specific_drug_sensitivity = state["models_results"]
        cumulative_cm = state["cumulative_cm"]
        all_fpr_test = state["all_fpr_test"]
        all_precision_test = state["all_precision_test"]
        drug_names = state["drug_names"]
        y_true_total = state["y_true_total"]
        y_pred_total = state["y_pred_total"]
        start_idx = len(models_results_rf_specific_drug_sensitivity)
    else:
        models_results_rf_specific_drug_sensitivity = {}
        cumulative_cm = np.array([[0, 0], [0, 0]])
        all_fpr_test = []
        all_precision_test = []
        drug_names = []
        y_true_total = []
        y_pred_total = []
        start_idx = 0  


    for specific_drug in tqdm(score_df["Drug"].iloc[start_idx+1:l], desc="Processing Drugs"):
        # Train model for each drug

        print(f"RAM used: {psutil.virtual_memory().used / (1024 ** 3):.2f} GB")
        # Directory
        models_dir = os.path.join(shap_dir, f"{drug_name}_models")
        os.makedirs(models_dir, exist_ok=True)
        
        df_drug = df_encoded[df_encoded["Drug_id"] == specific_drug]
        drug_name = score_df.loc[score_df["Drug"] == specific_drug, "Drug_name"].values[0]
        best_features = pd.read_csv(f'Results/Models_Sensitivity/RF/{drug_name}/top20_features.csv', index_col=0)
        cell_counts = df_drug["Cell_line_cosmic_identifiers"].value_counts()
        train_cells, test_cells = train_test_split(
            cell_counts.index,
            test_size=0.2,
            stratify=df_drug.groupby("Cell_line_cosmic_identifiers")["Sensitivity"].apply(lambda x: x.mean()),
            random_state=42
        )
        train_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(train_cells)]
        test_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(test_cells)]
        
        
            
        models_per_fold, y_val_all, y_val_pred_all = modello_deep_learning(best_features, train_set, train_cells, rf_params={}, 
               base_models=None, param_grid=None, n_splits=5, random_state=42 , model_type = model_type)


        # Save models
        for fold_idx, model_info in enumerate(models_per_fold):
            model_path = os.path.join(models_dir, f"model_fold_{fold_idx}.pkl")
            joblib.dump(model_info, model_path)
            print(f"saved in {model_path}")

        avg_auc_score = np.mean([m["auc_score"] for m in models_per_fold])
        avg_auc_pr = np.mean([m["auc_pr"] for m in models_per_fold])
        avg_f1_score = np.mean([m["f1_score"] for m in models_per_fold])

        X_test = test_set[best_features.index.tolist()]
        y_test = test_set["Sensitivity"]

        aucprs = np.array([m["auc_pr"] for m in models_per_fold])
        weights = aucprs / aucprs.sum()
        
        
        X_test = X_test.values if hasattr(X_test, "values") else X_test  
        if model_type == 'cnn':
                X_test = X_test.reshape(-1, X_test.shape[1], 1)  
                y_pred_test_prob = sum(weights[i] * models_per_fold[i]["model"].predict(X_test)[:, 0] for i in range(len(weights)))
        elif model_type == 'mlp':
                X_test = np.array(X_test).astype("float32")
                X_test = X_test.reshape((X_test.shape[0], -1))
                y_pred_test_prob = sum(
                                        weights[i] * models_per_fold[i]["model"].predict(X_test)
                                        for i in range(len(weights))
                                    )
                
        precision_val, recall_val, thresholds = precision_recall_curve(y_val_all, y_val_pred_all)
        f1_scores = 2 * (precision_val * recall_val) / (precision_val + recall_val + 1e-6)
        best_threshold = thresholds[np.argmax(f1_scores)]

        y_pred_test = (y_pred_test_prob > best_threshold).astype(int)

        auc_score_test = roc_auc_score(y_test, y_pred_test_prob)
        auc_pr_test = average_precision_score(y_test, y_pred_test_prob)
        f1_test = f1_score(y_test, y_pred_test)

        cutoff_range = np.arange(y_pred_test_prob.min(), y_pred_test_prob.max(), 0.01)
        precisions_cutoff = []
        recalls_cutoff = []

        true_labels = np.array(y_test)
        predicted_probabilities = np.array(y_pred_test_prob)

        for cutoff in cutoff_range:
            predicted_labels = (predicted_probabilities > cutoff).astype(int)
            precision = precision_score(true_labels, predicted_labels, zero_division=0)
            recall = recall_score(true_labels, predicted_labels, zero_division=0)
            precisions_cutoff.append(precision)
            recalls_cutoff.append(recall)

        precision_test_val = precision_score(y_test, y_pred_test)
        recall_test_val = recall_score(y_test, y_pred_test)

        y_true_total.extend(y_test)
        y_pred_total.extend(y_pred_test)

        fpr_val, tpr_val, _ = roc_curve(y_val_all, y_val_pred_all)
        fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_test_prob)
        precision_test, recall_test, _ = precision_recall_curve(y_test, y_pred_test_prob)

        all_fpr_test.append((fpr_test, tpr_test))
        all_precision_test.append((recall_test, precision_test))
        drug_names.append(specific_drug)

        # PR cutoff
        plt.figure(figsize=(7, 4))
        plt.plot(cutoff_range, precisions_cutoff, marker='o', label='Precision', color='blue')
        plt.plot(cutoff_range, recalls_cutoff, marker='x', label='Recall', color='orange')
        plt.title(f"Precision & Recall vs Probability Cut-off - {drug_name}")
        plt.xlabel("Probability Cut-off")
        plt.ylabel("Score")
        plt.ylim(0, 1.05)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"{shap_dir}/{drug_name}_precision_recall_vs_cutoff.png")
        plt.close()

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred_test)
        cumulative_cm += cm
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])
        cm_fig_path = os.path.join(shap_dir, f"{drug_name}_confusion_matrix.png")
        disp.plot(cmap="Blues")
        plt.title(f"Confusion Matrix - {drug_name}")
        plt.tight_layout()
        plt.savefig(cm_fig_path)
        plt.close()

        # Roc curve and PR-curve
        roc_fig_path = os.path.join(shap_dir, f"{drug_name}_roc_curve.png")
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(fpr_val, tpr_val, label=f'Validation AUC = {avg_auc_score:.3f}')
        plt.plot(fpr_test, tpr_test, label=f'Test AUC = {auc_score_test:.3f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {drug_name}")
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(recall_val, precision_val, label=f'Validation AUC-PR = {avg_auc_pr:.3f}')
        plt.plot(recall_test, precision_test, label=f'Test AUC-PR = {auc_pr_test:.3f}')
        plt.axvline(x=best_threshold, color='r', linestyle="--", label=f"Best Threshold = {best_threshold:.3f}")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Precision-Recall Curve - {drug_name}")
        plt.legend()

        # Print and save results
        models_results_rf_specific_drug_sensitivity[drug_name] = {
            "avg_auc_score": avg_auc_score,
            "avg_auc_pr": avg_auc_pr,
            "avg_f1_score": avg_f1_score,
            "auc_score_test": auc_score_test,
            "auc_pr_test": auc_pr_test,
            "f1_score_test": f1_test,
            "best_threshold": best_threshold,
            "precision_test": precision_test_val,
            "recall_test": recall_test_val
        }

        print(f"\n **Results for {drug_name}** ")
        print(f"AUC-ROC Test: {auc_score_test:.3f}")
        print(f"AUC-PR Test: {auc_pr_test:.3f}")
        print(f"F1-score Test: {f1_test:.3f}")
        print("\n **Validation Set Classification Report:**")
        print(classification_report(y_val_all, (np.array(y_val_pred_all) > 0.5).astype(int)))
        print("\n**Test Set Classification Report:**")
        print(classification_report(y_test, y_pred_test))

        # Save reuslts
        plt.tight_layout()
        plt.savefig(roc_fig_path)
        plt.close()
        tf.keras.backend.clear_session()
        gc.collect()
        state = {
            "models_results": models_results_rf_specific_drug_sensitivity,
            "cumulative_cm": cumulative_cm,
            "all_fpr_test": all_fpr_test,
            "all_precision_test": all_precision_test,
            "drug_names": drug_names,
            "y_true_total": y_true_total,
            "y_pred_total": y_pred_total
        }
        save_state(state_file, state)



    print("\n\n========== RIASSUNTO FINALE ==========")

    final_f1 = f1_score(y_true_total, y_pred_total)
    results_df = pd.DataFrame.from_dict(models_results_rf_specific_drug_sensitivity, orient="index")
    results_df.to_csv(os.path.join(shap_dir, "Drug_sensitivity_results.csv"))

    cm_fig_path = os.path.join(shap_dir, "total_confusion_matrix.png")
    disp = ConfusionMatrixDisplay(confusion_matrix=cumulative_cm, display_labels=["Negative", "Positive"])
    fig, ax = plt.subplots()
    disp.plot(cmap="Purples", ax=ax)
    plt.title(f"Confusion Matrix - TOTALE (Test Set)\nF1-score Finale: {final_f1:.4f}")
    plt.tight_layout()
    plt.savefig(cm_fig_path)
    plt.close()

    roc_all_fig_path = os.path.join(shap_dir, "roc_curve_all_drugs.png")
    plt.figure(figsize=(10, 5))
    for i, (fpr, tpr) in enumerate(all_fpr_test):
        plt.plot(fpr, tpr, label=drug_names[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve - Tutti i farmaci")
    plt.legend()
    plt.tight_layout()
    plt.savefig(roc_all_fig_path)
    plt.close()

    pr_all_fig_path = os.path.join(shap_dir, "precision_recall_curve_all_drugs.png")
    plt.figure(figsize=(10, 5))
    for i, (recall, precision) in enumerate(all_precision_test):
        plt.plot(recall, precision, label=drug_names[i])
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve - Tutti i farmaci")
    plt.legend()
    plt.tight_layout()
    plt.savefig(pr_all_fig_path)
    plt.close()



### Model MLP

In [ ]:
shap_dir = 'Results/Models_Sensitivity/mlp_Sensitivity'

train_model(df_clean, shap_dir, l= 31,  model_type = 'mlp')

### Model CNN

In [ ]:
shap_dir = 'Results/Models_Sensitivity/cnn_Sensitivity'

train_model(df_clean,shap_dir, l= 31, deeplearning = True, model_type = 'cnn')

## Model IC50

In this section, you can run models that predict IC50 and Drug sensitivity. The analysis specifically focuses on the top 30 drugs, although you can include more if desired. However, note that drugs with a negative score perform worse than a random model and are therefore not recommended for use—this applies especially to drugs ranked beyond the 194th position.

The DL functions are implemented here with the teo different approaches (SMOTE+KNN and no oversampling)

### Funzione

#### MLP

In [ ]:
def create_mlp_model_ic50(input_shape, dim=128, dropout=0.2, learning_rate = 0.01):

    """
    Creates and compiles a simple multi-layer perceptron (MLP) model for IC50 regression.
    
    The architecture includes three hidden layers with ReLU activation, dropout for regularization, 
    and a final linear output layer. Uses Adam optimizer with specified learning rate.
    
    Returns the model.
    """
    model = models.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    model.add(layers.Dense(dim, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dim//2, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(dim//4, activation='relu'))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model




def create_objective(X_train, X_val, y_train, y_val):
    """
    Creates an Optuna objective function to optimize hyperparameters for an MLP model.

    The objective trains a model with suggested hyperparameters (layer size, dropout, learning rate),
    evaluates it on validation data, and returns the average precision score as the optimization target.
    
    Returns the objective function to be used in an Optuna study.
    """

    def objective(trial):
        dim = trial.suggest_categorical('dim', [64, 128, 256, 512,1024])
        dropout = trial.suggest_float('dropout', 0.1, 0.6)
        learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

        try:

            model = tf.keras.Sequential([
                    tf.keras.Input(shape=(X_train.shape[1],)),
                    tf.keras.layers.Dense(dim, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(dim // 2, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(dim // 4, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(1, activation='linear')
                ])


            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)

            model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    verbose=0,
                    callbacks=[early_stopping])
            y_pred = model.predict(X_val, verbose = 0).ravel()
            r2 = r2_score(y_val, y_pred)

        except Exception as e:
            print(f"[ERROR] Optuna CNN Trial fallito: {e}")
            return -1000

        return r2

    return objective


#### CNN

In [ ]:
def create_cnn_model_ic50(input_shape, dim=128, learning_rate = 0.01):
    """
    Creates and compiles a 1D convolutional neural network (CNN) model for IC50 regression.

    The model consists of three Conv1D layers with ReLU activations and max pooling, followed by
    a fully connected layer and a sigmoid output layer. Uses Adam optimizer with given learning rate.

    Returns the compiled CNN model.
    """
    model = models.Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    model.add(layers.Conv1D(dim, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(dim//2, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(dim//4, 3, activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(dim//8, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model


def create_objective_cnn(X_train, X_val, y_train, y_val):
    """
    Creates an Optuna objective function to optimize hyperparameters for the 1D CNN model.

    The objective trains a CNN with trial-suggested parameters (dim and learning rate),
    performs early stopping, and evaluates performance on validation data using average precision score.

    Skips trials if input length is too short for the CNN architecture.

    Returns the objective function for use in an Optuna study.
    """

    def objective(trial):
        dim = trial.suggest_categorical('dim', [64, 128, 256, 512])
        learning_rate = trial.suggest_float('lr', 1e-5, 1e-2, log=True)

        try:

            model = tf.keras.Sequential([
                tf.keras.Input(shape=(X_train.shape[1], 1)),
                tf.keras.layers.Conv1D(dim, 3, activation='relu'),
                tf.keras.layers.MaxPooling1D(2),
                tf.keras.layers.Conv1D(dim//2, 3, activation='relu'),
                tf.keras.layers.MaxPooling1D(2),
                tf.keras.layers.Conv1D(dim//4, 3, activation='relu'),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(dim//8, activation='relu'),
                tf.keras.layers.Dense(1, activation='linear')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
            model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0)

            model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    verbose=0,
                    callbacks=[early_stopping])

            y_pred = model.predict(X_val, verbose=0).ravel()

        except Exception as e:
            print(f"[ERROR] Optuna CNN Trial fallito: {e}")
            return -1000
        
        return r2_score(y_val, y_pred)

    return objective




#### Model

In [ ]:
def modello_deep_learning_ic50(best_features, train_set, train_cells, rf_params={}, 
               base_models=None, param_grid=None, n_splits=5, random_state=4, model_type = 'mlp', smo = False):
    
    """
    Performs k-fold cross-validation training of deep learning models (MLP or CNN) on the given dataset.

    For each fold:
    - Splits data into training and validation based on cell line identifiers.
    - Applies SMOTE if True to balance classes in training data.
    - Reshapes data as needed for the selected model type.
    - Uses Optuna to optimize hyperparameters (architecture dimensions, dropout, learning rate).
    - Trains the model with early stopping on validation loss.
    - Evaluates and stores performance metrics (ROC AUC, AUC PR, F1 score).
    
    Returns a list of trained models with their metrics, and predicted results.
    """
    
    # Base models
    if base_models is None:
        base_models = [create_mlp_model_ic50 for _ in range(n_splits)]  
    
    #KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    models_per_fold = []
    y_scalers_per_fold = []
    y_val_all, y_val_pred_all, y_sens_all = [], [], []
    i = 0

    for train_idx, val_idx in kf.split(train_cells):
        # Split training and validation
        train_cells_fold = train_cells[train_idx]
        val_cells_fold = train_cells[val_idx]
        train_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(train_cells_fold)]
        val_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(val_cells_fold)]
        X_train = train_fold[best_features.index.tolist()]
        y_train = train_fold["IC50"]
        y_train_sens = train_fold["Sensitivity"]
        X_val = val_fold[best_features.index.tolist()]
        y_val = val_fold["IC50"]
        y_sens = val_fold["Sensitivity"]

        if len(train_fold) == 0 or len(val_fold) == 0:
            print(f"Fold {i}: train or validation fold is empty. Skipping.")
            continue

        if smo:
                # SMOTE
                smote = SMOTE(random_state=42)
                X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train_sens)

                knn = KNeighborsRegressor(n_neighbors=3)
                knn.fit(X_train, y_train)

                y_ic50_resampled = knn.predict(X_train_resampled)
                df_train_final = X_train_resampled.copy()
                df_train_final["IC50"] = y_ic50_resampled
                df_train_final["Sensitivity"] = y_train_resampled

                X_train_smo = df_train_final.drop(columns=["IC50"])
                y_train_ic50_smo = df_train_final["IC50"]

                X_train = pd.DataFrame(X_train_smo, columns=X_train.columns)
                y_train = pd.Series(y_train_ic50_smo, index=X_train.index)

                y_scaler = StandardScaler()
                y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
                y_val = y_scaler.transform(y_val.values.reshape(-1, 1)).ravel()

        else: 
            y_scaler = StandardScaler()
            y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
            y_val = y_scaler.transform(y_val.values.reshape(-1, 1)).ravel()

        if model_type == 'cnn':
            X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
            X_val = X_val.values.reshape(-1, X_val.shape[1], 1)
        else: 
            X_train = X_train.values.astype("float32")
            X_val = X_val.values.astype("float32")


        # Create models and Hyp.Tuning
        if model_type == 'mlp':
            objective = create_objective(X_train, X_val, y_train, y_val)
            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=10)

            print("Best hyperparameters found:")
            print(study.best_params)

            best_params = study.best_params
            model = create_mlp_model_ic50(
                input_shape=(X_train.shape[1],),
                dim=best_params['dim'],
                dropout=best_params['dropout'],
                learning_rate=best_params['lr']
            )
 
        elif model_type == 'cnn':
            objective = create_objective_cnn(X_train, X_val, y_train, y_val)
            study = optuna.create_study(direction='maximize')
            study.optimize(objective, n_trials=10)

            print("Best hyperparameters found:")
            print(study.best_params)

            best_params = study.best_params
            model = create_cnn_model_ic50(
                input_shape=(X_train.shape[1],1),
                dim=best_params['dim'],
                learning_rate=best_params['lr']
            )

            
        
        # Model fit
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1)
        model.fit(X_train, y_train, 
                  epochs=20, 
                  batch_size=min(len(X_train), 32), 
                  verbose=0, 
                  validation_data=(X_val, y_val), 
                  callbacks=[early_stopping])
        
        # Predict IC50
        y_pred_ic50 = model.predict(X_val, verbose = 0)
        y_pred_ic50 = y_scaler.inverse_transform(y_pred_ic50)
        y_scalers_per_fold.append(y_scaler)
        y_val_all.extend(y_val)
        y_val_pred_all.extend(y_pred_ic50)
        y_sens_all.extend(y_sens)

        models_per_fold.append(model)

        i += 1

    return models_per_fold, y_sens_all, y_val_all, y_val_pred_all,y_scalers_per_fold


#### Testing

In [ ]:
def testing(best_features, train_set, train_cells,shap_dir, drug_name,model_type, n_splits=5, random_state=4, smo = False):
    """
    Tests previously trained models for a specific drug on validation folds and evaluates their performance.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    models_per_fold = []
    y_scalers_per_fold = []
    y_val_all, y_val_pred_all, y_sens_all = [], [], []
    i = 0
    models_dir = os.path.join(shap_dir, f"{drug_name}_models")

    for train_idx, val_idx in kf.split(train_cells):
        # Model
        model_path = os.path.join(models_dir, f"model_fold_{i}.pkl")
        model = joblib.load(model_path)
        models_per_fold.append(model)

        # Split data
        train_cells_fold = train_cells[train_idx]
        val_cells_fold = train_cells[val_idx]
        train_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(train_cells_fold)]
        val_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(val_cells_fold)]
        X_train = train_fold[best_features.index.tolist()]
        y_train = train_fold["IC50"]
        y_train_sens = train_fold["Sensitivity"]
        X_val = val_fold[best_features.index.tolist()]
        y_val = val_fold["IC50"]
        y_sens = val_fold["Sensitivity"]

        if len(train_fold) == 0 or len(val_fold) == 0:
            print(f"Fold {i}: train or validation fold is empty. Skipping.")
            continue

        if smo:
                # SMOTE
                smote = SMOTE(random_state=42)
                X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train_sens)

                knn = KNeighborsRegressor(n_neighbors=3)
                knn.fit(X_train, y_train)

                y_ic50_resampled = knn.predict(X_train_resampled)
                df_train_final = X_train_resampled.copy()
                df_train_final["IC50"] = y_ic50_resampled
                df_train_final["Sensitivity"] = y_train_resampled

                X_train_smo = df_train_final.drop(columns=["IC50"])
                y_train_ic50_smo = df_train_final["IC50"]

                X_train = pd.DataFrame(X_train_smo, columns=X_train.columns)
                y_train = pd.Series(y_train_ic50_smo, index=X_train.index)

                y_scaler = StandardScaler()
                y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
                y_val = y_scaler.transform(y_val.values.reshape(-1, 1)).ravel()

        else: 
            y_scaler = StandardScaler()
            y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
            y_val = y_scaler.transform(y_val.values.reshape(-1, 1)).ravel()

        if model_type == 'cnn':
            X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
            X_val = X_val.values.reshape(-1, X_val.shape[1], 1)
        else: 
            X_train = X_train.values.astype("float32")
            X_val = X_val.values.astype("float32")



        # Predict IC50
        y_pred_ic50 = model.predict(X_val, verbose = 0)
        y_pred_ic50 = y_scaler.inverse_transform(y_pred_ic50)
        y_scalers_per_fold.append(y_scaler)
        y_val_all.extend(y_val)
        y_val_pred_all.extend(y_pred_ic50)
        y_sens_all.extend(y_sens)

        i += 1

    return models_per_fold, y_sens_all, y_val_all, y_val_pred_all,y_scalers_per_fold


#### train model

In [ ]:
def train_model_ic50(df_clean, shap_dir, l=31, model_type = 'mlp', smo = False):
    """
    Trains predictive models for drug IC50 and sensitivity.
    The function processes multiple drugs in a loop, performing data preprocessing, training, validation, and evaluation 
    for each drug separately.

    Key steps include:
    - Loading a previous training state to allow resuming interrupted training
    - Splitting data into training and test sets stratified by sensitivity
    - Training models with K-fold cross-validation and hyperparameter tuning via Optuna
    - Aggregating results and calculating performance metrics such as AUC-ROC, AUC-PR, and F1-score
    - Generating and saving plots (precision-recall vs cutoff, confusion matrix, ROC and PR curves)
    - Saving intermediate training states for checkpointing and resuming

    Parameters:
    - df_clean: cleaned input dataframe containing features and drug response labels
    - shap_dir: directory path to save results, plots, and model checkpoints
    - l: number of drugs to process (default 31)
    - model_type: specifies the type of deep learning model ('mlp' or 'cnn')
    - smo True if oversampling


    """
    # Directory
    if not os.path.exists(shap_dir):
        os.makedirs(shap_dir)

    label_encoders = {}
    df_encoded = df_clean.copy()
    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object' or df_encoded[col].dtype.name == 'category':
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le

    df_nan_sens = df_encoded[df_encoded["Sensitivity"].isna()]  
    df_encoded = df_encoded[df_encoded["Sensitivity"].notna()]
    state_file = os.path.join(shap_dir, "training_state.pkl")
    state = load_state(state_file)

    if not state:
        print("Stato precedente trovato. Riprendo da dove avevo interrotto.")
        models_results_rf_ic50 = state["models_results"]
        y_test_all = state["y_test_all"]
        y_pred_all = state["y_pred_all"]
        y_test_sens_all = state["y_test_sens_all"]
        y_pred_sens_all = state["y_pred_sens_all"]
        sensitivity_all = state["sensitivity_all"]
        start_idx = len(models_results_rf_ic50)
    else:
        models_results_rf_ic50 = {}
        y_test_all = []
        y_pred_all = []
        y_test_sens_all =[]
        y_pred_sens_all = []
        sensitivity_all =[]
        start_idx = 0


    for specific_drug in tqdm(score_df["Drug"].iloc[start_idx+1:l], desc="Processing Drugs"):
        # Model for each drug
        df_drug = df_encoded[df_encoded["Drug_id"] == specific_drug]
        drug_name = score_df.loc[score_df["Drug"] == specific_drug, "Drug_name"].values[0]
        best_features = pd.read_csv(f'Results/Models_Sensitivity/RF/{drug_name}/top20_features.csv', index_col=0)

        #SPli training and test data
        cell_counts = df_drug["Cell_line_cosmic_identifiers"].value_counts()
        train_cells, test_cells = train_test_split(
            cell_counts.index,
            test_size=0.2,
            stratify=df_drug.groupby("Cell_line_cosmic_identifiers")["Sensitivity"].apply(lambda x: x.mean()),
            random_state=42
        )
        train_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(train_cells)]
        test_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(test_cells)]
        X_test = test_set[best_features.index.tolist()]
        y_test_ic50 = test_set["IC50"]
        sensitivities = test_set["Sensitivity"]

        '''
        # If you want to train the model again
        models_per_fold, y_val_sens, y_val_ic50, y_pred_val,y_scalers_per_fold = modello_deep_learning_ic50(best_features, train_set, train_cells, rf_params={}, 
               base_models=None, param_grid=None, n_splits=5, random_state=42 , model_type = model_type, smo = smo)


        models_dir = os.path.join(shap_dir, f"{drug_name}_models")
        os.makedirs(models_dir, exist_ok=True)

        for fold_idx, model_info in enumerate(models_per_fold):
            model_path = os.path.join(models_dir, f"model_fold_{fold_idx}.pkl")
            joblib.dump(model_info, model_path)
            print(f"saved in {model_path}")
            

        '''
        # Model already trained
        models_per_fold, y_val_sens, y_val_ic50, y_pred_val,y_scalers_per_fold = testing(best_features, train_set, train_cells,shap_dir, drug_name,model_type, n_splits=5, random_state=4, smo = False)
        

        X_test = X_test.values if hasattr(X_test, "values") else X_test  
        if model_type == 'cnn':
                X_test = X_test.reshape(-1, X_test.shape[1], 1)  
                y_pred_test_ic50 = sum(1/5 * models_per_fold[i].predict(X_test)[:, 0] for i in range(5))
        elif model_type == 'mlp':
                X_test = np.array(X_test).astype("float32")
                X_test = X_test.reshape((X_test.shape[0], -1))
                y_pred_test_ic50 = sum(
                                        1/5 * models_per_fold[i].predict(X_test)
                                        for i in range(5)
                                    )
     

        # PREDICT IC50
        y_pred_test_ic50 = np.mean([m.predict(X_test) for m in models_per_fold], axis=0)
        y_pred_test_ic50 = y_scalers_per_fold[0].inverse_transform(y_pred_test_ic50)

        # Metrics
        mae = mean_absolute_error(y_test_ic50, y_pred_test_ic50)
        rmse = np.sqrt(mean_squared_error(y_test_ic50, y_pred_test_ic50))
        r2 = r2_score(y_test_ic50, y_pred_test_ic50)
        f1 = f1_score(true_sensitivity, y_pred_sensitivity)
        pearson_corr, _ = pearsonr(y_test_ic50, y_pred_test_ic50.ravel())

        df_plot = pd.DataFrame({
                                "True_IC50": np.array(y_test_ic50).ravel(),
                                "Predicted_IC50": y_pred_test_ic50.ravel(),
                                "Sensitive": sensitivities.map({0: "Non-sensitive", 1: "Sensitive"})
                            })
        # Cutoff
        cutoff_range = np.arange(y_pred_test_ic50.min(), y_pred_test_ic50.max(), 0.1)
        precisions_cutoff = []
        recalls_cutoff = []
        true_sensitivity = np.array(sensitivities)
        y_pred_test_ic50 = np.array(y_pred_test_ic50)

        for cutoff in cutoff_range:
            predicted_sensitivity = (y_pred_test_ic50 < cutoff).astype(int)
            precision = precision_score(true_sensitivity, predicted_sensitivity, zero_division=0)
            recall = recall_score(true_sensitivity, predicted_sensitivity, zero_division=0)

            precisions_cutoff.append(precision)
            recalls_cutoff.append(recall)

        # Best threshold 
        y_score_test = -y_pred_test_ic50  
        y_score_val =  -np.array(y_pred_val)
        fpr_test, tpr_test, roc_thresholds = roc_curve(sensitivities, y_score_test)
        fpr_val, tpr_val, roc_thresholds_val = roc_curve(y_val_sens, y_score_val)
        auc_score_test = roc_auc_score(sensitivities, y_score_test)
        auc_score_val = roc_auc_score(y_val_sens, y_score_val)
        precision_test, recall_test, pr_thresholds = precision_recall_curve(sensitivities, y_score_test)
        avg_auc_pr_test = average_precision_score(sensitivities, y_score_test)
        precision_val, recall_val, pr_thresholds_val = precision_recall_curve(y_val_sens, y_score_val)
        avg_auc_pr_val = average_precision_score(y_val_sens, y_score_val)
        f1_scores_val = 2 * (precision_val[:-1] * recall_val[:-1]) / (precision_val[:-1] + recall_val[:-1] + 1e-8)
        best_idx_val = np.argmax(f1_scores_val)
        best_threshold_val = -pr_thresholds_val[best_idx_val]
        y_pred_sensitivity = (y_pred_test_ic50 <= best_threshold_val).astype(int)


        y_test_all.extend(y_test_ic50)
        y_pred_all.extend(y_pred_test_ic50)
        y_test_sens_all.extend(true_sensitivity)
        y_pred_sens_all.extend(y_pred_sensitivity)
        sensitivity_all.extend(sensitivities)

        # Save results
        models_results_rf_ic50[specific_drug] = {
                "MAE_IC50": mae,
                "RMSE_IC50": rmse,
                "R2_IC50": r2,
                "Pearson": pearson_corr,
                "F1-score": f1,
                "ROC_AUC": auc_score_test,
                "PR_AUC": avg_auc_pr_test,
            }
        

        # PLot IC50
        plt.figure(figsize=(6, 6))
        sns.scatterplot(data=df_plot, x="True_IC50", y="Predicted_IC50", hue="Sensitive", alpha=0.7)
        plt.plot([df_plot["True_IC50"].min(), df_plot["True_IC50"].max()],
                [df_plot["True_IC50"].min(), df_plot["True_IC50"].max()], 'r--')
        plt.xlabel("True IC50")
        plt.ylabel("Predicted IC50")
        plt.title(f"IC50 Prediction- {drug_name}")
        plt.legend(title="Cell Sensitivity")
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{shap_dir}/{drug_name}_ic50_scatter_rf.png")
        plt.close()

        #PR cutoff
        plt.figure(figsize=(7, 4))
        plt.plot(cutoff_range, precisions_cutoff, marker='o', label='Precision', color='blue')
        plt.plot(cutoff_range, recalls_cutoff, marker='x', label='Recall', color='orange')
        plt.title(f"Precision & Recall vs IC50 Cut-off - {drug_name}")
        plt.xlabel("IC50 Cut-off (on predicted values)")
        plt.ylabel("Score")
        plt.ylim(0, 1.05)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"{shap_dir}/{drug_name}_precision_recall_curve.png")
        plt.close()

        
        # Confusion Matrix
        cm = confusion_matrix(true_sensitivity, y_pred_sensitivity)
        plt.figure(figsize=(4, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Non-sensitive", "Sensitive"], yticklabels=["Non-sensitive", "Sensitive"])
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title(f"Confusion Matrix with cutoff- {drug_name}")
        plt.tight_layout()
        plt.savefig(f"{shap_dir}/{drug_name}_confusion_matrix.png")
        plt.close()

        # ROc curve
        roc_fig_path = os.path.join(shap_dir, f"{drug_name}_roc_curve.png")
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(fpr_val, tpr_val, label=f'Validation AUC = {auc_score_test:.3f}')
        plt.plot(fpr_test, tpr_test, label=f'Test AUC = {auc_score_test:.3f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {drug_name}")
        plt.legend()
        plt.subplot(1, 2, 2)
        plt.plot(recall_val, precision_val, label=f'Validation AUC-PR = {avg_auc_pr_val:.3f}')
        plt.plot(recall_test, precision_test, label=f'Test AUC-PR = {avg_auc_pr_test:.3f}')
        plt.axvline(x=recall_val[best_idx_val], color='r', linestyle="--", label=f"Best Threshold = {best_threshold_val:.3f}")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Precision-Recall Curve - {drug_name}")
        plt.legend()
        plt.tight_layout()
        plt.savefig(roc_fig_path)
        plt.close()


        # Print and save results
        print(f"\n** Results for {drug_name} **")
        print(f"IC50 - MAE: {mae:.3f}, RMSE: {rmse:.3f}, R²: {r2:.3f}")

        tf.keras.backend.clear_session()
        gc.collect()
        state = {
            "models_results": models_results_rf_ic50,
            "y_test_all" :y_test_all,
            "y_pred_all":y_pred_all,
            "y_test_sens_all" :y_test_sens_all,
            "y_pred_sens_all":y_pred_sens_all,
            "sensitivity_all" : sensitivity_all,
            "start_idx" : start_idx
        }
        save_state(state_file, state)

    
    results_df = pd.DataFrame.from_dict(models_results_rf_ic50, orient="index")
    results_df.to_csv(os.path.join(shap_dir, "rf_drug_sensitivity_results.csv"))

    # Final metrics
    mae_global = mean_absolute_error(y_test_all, y_pred_all)
    rmse_global = np.sqrt(mean_squared_error(y_test_all, y_pred_all))
    r2_global = r2_score(y_test_all, y_pred_all)
    final_f1 = f1_score(y_test_sens_all, y_pred_sens_all)
    cm_global = confusion_matrix(y_test_sens_all, y_pred_sens_all)

    for i, val in enumerate(y_test_sens_all):
        if not np.isscalar(val):
            print(f"Non-scalar at index {i}: {val} ({type(val)})")

    df_global = pd.DataFrame({
        "True_IC50": y_test_all,
        "Predicted_IC50": y_pred_all,
        "Sensitive": pd.Series([int(x) for x in y_test_sens_all]).map({0: "Non-sensitive", 1: "Sensitive"})

    })

    df_global["Predicted_IC50"] = df_global["Predicted_IC50"].apply(lambda x: float(x[0]) if isinstance(x, (np.ndarray, list)) else float(x))



    print("\n\n========= GLOBAL METRICS ACROSS ALL DRUGS =========")
    print(f"Global IC50 - MAE: {mae_global:.3f}, RMSE: {rmse_global:.3f}, R²: {r2_global:.3f}")


    # Confusion matrix
    plt.figure(figsize=(5, 5))
    sns.heatmap(cm_global, annot=True, fmt='d', cmap='Purples', xticklabels=["Non-sensitive", "Sensitive"], yticklabels=["Non-sensitive", "Sensitive"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Global Confusion Matrix (All Drugs) with f1: {final_f1}")
    plt.tight_layout()
    plt.savefig(f"{shap_dir}/global_confusion_matrix_rf.png")
    plt.close()


    #FInal IC50
    plt.figure(figsize=(7, 7))
    sns.scatterplot(data=df_global, x="True_IC50", y="Predicted_IC50", hue="Sensitive", alpha=0.5)
    plt.plot([df_global["True_IC50"].min(), df_global["True_IC50"].max()],
            [df_global["True_IC50"].min(), df_global["True_IC50"].max()], 'r--')
    plt.xlabel("True IC50")
    plt.ylabel("Predicted IC50")
    plt.title(f"Global IC50 Prediction Across All Drugs R²: {r2_global:.3f}")
    plt.legend(title="Cell Sensitivity")
    plt.grid(True)
    plt.tight_layout()
    plt.tight_layout()
    plt.savefig(f"{shap_dir}/global_ic50_prediction_rf.png")
    plt.close()


### NO versampling

#### CNN

In [ ]:
shap_dir = 'Results/Models_IC50/cnn'

train_model_ic50(df_clean, shap_dir, l= 31, model_type = 'cnn')

#### MLP

In [ ]:
shap_dir = 'Results/Models_IC50/mlp'

train_model_ic50(df_clean,shap_dir, l= 31, model_type = 'mlp')

### Oversampling

#### MLP

In [ ]:
shap_dir = 'Results/Models_IC50_SMOTE/mlp'

train_model_ic50(df_clean,shap_dir, l= 31, model_type = 'mlp', smo = True)

#### CNN

In [ ]:
shap_dir = 'Results/Models_IC50_SMOTE/cnn'

train_model_ic50(df_clean,shap_dir, l= 31,  model_type = 'cnn', smo = True)